
# Demo: Convoluciones clásicas y una CNN mínima en PyTorch

Este cuaderno muestra:
1. Cómo **aplicar kernels (filtros)** clásicos mediante convolución 2D en PyTorch (Sobel, Laplaciano, Blur, Sharpen, Emboss, etc.).  
2. Una **CNN mínima** y la **visualización de mapas de activación** (feature maps) de su primera capa para una imagen de ejemplo.  
3. **(Opcional)** un entrenamiento rápido sobre MNIST para comparar activaciones **antes** y **después** del entrenamiento.

> Sugerencias docentes: cambia los valores de los kernels y observa el efecto, combina filtros (e.g., Blur → Sobel), y modifica tamaño de kernel/`padding`/`stride`.


## Importaciones y utilidades

In [ ]:

%matplotlib inline
import os, math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# ----------------------------
# Utilidades de visualización
# ----------------------------
def imshow_gray(t, title=None):
    """
    Muestra un tensor imagen [H,W] o [1,H,W] en escala de grises.
    """
    if isinstance(t, torch.Tensor):
        tt = t
    else:
        tt = torch.as_tensor(t)
    if tt.dim() == 3:
        tt = tt.squeeze(0)
    tt = tt.detach().cpu().numpy()
    plt.figure()
    plt.imshow(tt, cmap='gray', vmin=tt.min(), vmax=tt.max())
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

def grid_show(tensor_list, titles=None, ncols=3, suptitle=None):
    """
    Muestra una lista de tensores [H,W] o [1,H,W] en una grilla.
    """
    n = len(tensor_list)
    ncols = min(ncols, n)
    nrows = math.ceil(n / ncols)
    import numpy as _np
    fig, axes = plt.subplots(nrows, ncols, figsize=(3*ncols, 3*nrows))
    if nrows == 1 and ncols == 1:
        axes = _np.array([[axes]])
    elif nrows == 1:
        axes = _np.array([axes])

    for i, ax in enumerate(axes.flat):
        if i < n:
            t = tensor_list[i]
            if isinstance(t, torch.Tensor):
                tt = t
            else:
                tt = torch.as_tensor(t)
            if tt.dim() == 3:
                tt = tt.squeeze(0)
            ax.imshow(tt.detach().cpu().numpy(), cmap='gray')
            ax.axis('off')
            if titles and i < len(titles):
                ax.set_title(titles[i])
        else:
            ax.axis('off')
    if suptitle:
        fig.suptitle(suptitle)
    plt.tight_layout()
    plt.show()


## Imagen sintética de prueba

In [ ]:

import numpy as np
import torch

def synthetic_checkerboard(H=128, W=128, block=8):
    """Tablero de cuadros = buen patrón para ver bordes."""
    yy, xx = np.indices((H, W))
    board = ((yy // block + xx // block) % 2).astype(np.float32)
    return torch.from_numpy(board)  # [H, W], 0/1

def synthetic_gradient_circle(H=128, W=128, r0=50):
    """Círculo suave sobre gradiente para ver realces."""
    yy, xx = np.indices((H, W))
    cy, cx = H//2, W//2
    dist = np.sqrt((yy-cy)**2 + (xx-cx)**2)
    circle = (dist < r0).astype(np.float32)
    gradx = (xx / (W-1)).astype(np.float32)
    base = 0.6*gradx + 0.4*circle
    return torch.from_numpy(base)

# Combinamos ambas para una escena simple:
img1 = synthetic_checkerboard(128, 128, block=8)  # [H,W]
img2 = synthetic_gradient_circle(128, 128, r0=38) # [H,W]
img = 0.5*img1 + 0.5*img2
img = (img - img.min()) / (img.max() - img.min())
x = img.unsqueeze(0).unsqueeze(0)  # [B=1, C=1, H, W]

imshow_gray(img, title="Imagen sintética (grises)")


## Definición y aplicación de kernels 3×3

In [ ]:

import torch.nn.functional as F

def make_kernel(array2d):
    k = torch.tensor(array2d, dtype=torch.float32)
    k = k.unsqueeze(0).unsqueeze(0)  # [out_channels=1, in_channels=1, kH, kW]
    return k

# Kernels clásicos 3x3
identity = make_kernel([[0,0,0],
                        [0,1,0],
                        [0,0,0]])

blur = (1/9.0) * make_kernel([[1,1,1],
                              [1,1,1],
                              [1,1,1]])

sharpen = make_kernel([[0,-1, 0],
                       [-1,5,-1],
                       [0,-1, 0]])

edge_laplacian = make_kernel([[0, 1, 0],
                              [1,-4, 1],
                              [0, 1, 0]])

sobel_x = make_kernel([[-1,0,1],
                       [-2,0,2],
                       [-1,0,1]])

sobel_y = make_kernel([[-1,-2,-1],
                       [ 0, 0, 0],
                       [ 1, 2, 1]])

emboss = make_kernel([[-2,-1, 0],
                      [-1, 1, 1],
                      [ 0, 1, 2]])

kernels = {
    "Identity": identity,
    "Blur (3x3)": blur,
    "Sharpen": sharpen,
    "Laplacian": edge_laplacian,
    "Sobel X": sobel_x,
    "Sobel Y": sobel_y,
    "Emboss": emboss,
}

def apply_kernel(x, k):
    # x: [1,1,H,W], k: [1,1,kH,kW]
    y = F.conv2d(x, k, padding='same')  # PyTorch >= 2.0
    # Normalizar para visualizar mejor:
    y = (y - y.min()) / (y.max() - y.min() + 1e-8)
    return y.squeeze(0).squeeze(0)

filtered = []
titles = []
for name, k in kernels.items():
    y = apply_kernel(x, k)
    filtered.append(y)
    titles.append(name)

grid_show([img] + filtered, titles=["Original"] + titles, ncols=4,
          suptitle="Convolución con kernels clásicos")


## CNN mínima y visualización de mapas de activación

In [ ]:

class SmallCNN(nn.Module):
    def __init__(self, n_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)   # 1->8
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)  # 8->16
        self.pool = nn.MaxPool2d(2,2)  # reduce H,W a la mitad
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16*7*7, 64),
            nn.ReLU(),
            nn.Linear(64, n_classes)
        )

    def forward(self, z):
        z = F.relu(self.conv1(z))  # [B,8,H,W]
        z = self.pool(z)           # [B,8,H/2,W/2]
        z = F.relu(self.conv2(z))  # [B,16,H/2,W/2]
        z = self.pool(z)           # [B,16,H/4,W/4]
        z = self.head(z)
        return z

# Redimensionamos la imagen sintética a 28x28 (estilo MNIST) para pasarla por la CNN
img28 = F.interpolate(x, size=(28,28), mode='bilinear', align_corners=False)  # [1,1,28,28]
net = SmallCNN(n_classes=10).eval()

with torch.no_grad():
    a1 = F.relu(net.conv1(img28))  # [1, 8, 28, 28]

# Visualizamos los 8 mapas (canales) de la primera conv
chans = a1.squeeze(0)  # [8, 28, 28]
grid_show([chans[i] for i in range(chans.size(0))],
          titles=[f"Conv1 map {i}" for i in range(chans.size(0))],
          ncols=4,
          suptitle="Mapas de activación - Conv1 (sin entrenamiento)")



## (Opcional) Entrenamiento rápido en MNIST

Ejecuta la celda para entrenar 1–2 épocas. Requiere `torchvision`.  
Al final, visualiza de nuevo los mapas de activación de `conv1` para comparar **antes vs. después**.


In [ ]:

def train_quick_mnist(epochs=1, batch_size=128, lr=1e-3, device='cpu'):
    try:
        import torchvision
        from torchvision import transforms
    except ImportError:
        print("torchvision no está instalado. Omite el entrenamiento opcional.")
        return None, None

    device = torch.device(device)
    tfm = transforms.Compose([transforms.ToTensor()])  # [0,1], [C,H,W]
    train_set = torchvision.datasets.MNIST(root="./data", train=True, transform=tfm, download=True)
    test_set  = torchvision.datasets.MNIST(root="./data", train=False, transform=tfm, download=True)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader  = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

    model = SmallCNN(n_classes=10).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    def eval_acc():
        model.eval()
        correct = 0; total = 0
        with torch.no_grad():
            for xx, yy in test_loader:
                xx, yy = xx.to(device), yy.to(device)
                logits = model(xx)
                pred = logits.argmax(1)
                correct += (pred == yy).sum().item()
                total += yy.numel()
        return correct/total if total else 0.0

    print("Entrenando (rápido) MNIST...")
    before = eval_acc()
    print(f"Accuracy inicial: {before:.3f}")

    model.train()
    for ep in range(epochs):
        for xx, yy in train_loader:
            xx, yy = xx.to(device), yy.to(device)
            opt.zero_grad(set_to_none=True)
            logits = model(xx)
            loss = loss_fn(logits, yy)
            loss.backward()
            opt.step()
        acc = eval_acc()
        print(f"Época {ep+1}/{epochs} - Accuracy: {acc:.3f}")

    return model, test_set

# Descomenta para entrenar y visualizar activaciones después del entrenamiento:
# model_trained, test_set = train_quick_mnist(epochs=2, device='cpu')
# if model_trained is not None:
#     model_trained.eval()
#     # Tomamos una imagen del test y miramos Conv1
#     x_ex, y_ex = test_set[0]
#     with torch.no_grad():
#         a1_after = F.relu(model_trained.conv1(x_ex.unsqueeze(0)))
#     chans_after = a1_after.squeeze(0)
#     grid_show([chans_after[i] for i in range(chans_after.size(0))],
#               titles=[f"Conv1 map {i} (entrenado)" for i in range(chans_after.size(0))],
#               ncols=4,
#               suptitle="Mapas de activación - Conv1 (después de entrenar)")



---

### Notas

- En `apply_kernel`, se usa `padding='same'` (disponible en PyTorch ≥ 2.0). Si usas una versión anterior, reemplaza por `padding=1` para kernels 3×3.  
- Para comparar PyTorch vs OpenCV, puedes replicar la sección de kernels con `cv2.filter2D` y observar diferencias numéricas (por normalización y bordes).  
- Para ver los **filtros aprendidos**, inspecciona `net.conv1.weight` antes y después de entrenar con MNIST.
